## **DQN Agent**

In [ ]:
import os
# no mostrar alertas de tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import gym
import math
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import deque

### **Declaración del agente**

In [ ]:
class DQNAgent:
    def __init__(self, environment, path, verbose=False):
        """ Inicializar el agente

        Args:
            environment (gym.wrappers.time_limit.TimeLimit): ambiente en el que se desenvuelve el agente
            path (string): ruta para almecenar/cargar la matriz de pesos para la NN
            verbose (bool, optional): habilitar los comentarios. Defaults to False.
        """
        # resguardamos el ambiente
        self._env = environment
        self._stateSize = self._env.observation_space.shape[0]
        self._actionSize = self._env.action_space.n

        # parámetros que decaerán con el tiempo
        self._epsilon = 1.0
        self._lr = 0.001
        self._gamma = 0.95
        # constantes para decaer los parámetros
        self._minEpsilon = 0.1
        self._minLr = 0.1
        self._discount = 0.98
        self._decay = 25

        # reservamos memoria para almacenar los estados requeridos para el
        # aprendizaje y generamos el modelo de NN que actuará como nuestra "matriz Q"
        self._memory  = deque(maxlen=2000)
        self._QValues = self._build_model()

        # otras cosas que requerimos
        self._path = path
        self._verbose = verbose


    def _build_model(self):
        """ Se define el modelo de red neuronal que se empleará para el aprendizaje
            del agente.

        Returns:
            keras.models.Sequential: modelo definido
        """
        model = keras.models.Sequential()
        model.add(keras.layers.Dense(5, input_dim=self._stateSize, activation="relu"))
        model.add(keras.layers.Dense(10, activation="relu"))
        model.add(keras.layers.Dense(self._actionSize, activation="linear"))
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=self._lr))
        return model


    def _decayEpsilon(self, t):
        """ calcular el decaimiento del exploration rate

        Args:
            t (int): instante de tiempo transcurrido durante el apisodio
        """
        self._epsilon = max(self._minEpsilon, min(1., 1. - math.log10((t + 1) / self._decay)))


    def _mapState(self, state):
        """ Transformar el estado continuo en un estado discreto mediante el empleo de buckets

        Args:
            state (tuple): el estado actual del ambiente

        Returns:
            list: el estado transformado
        """
        return np.reshape(state, [1, self._stateSize])


    def _capacity(self):
        """ Entregar la capacidad actual de nuestro buffer de memoria

        Returns:
            int: capacidad actual de nuestro buffer de memoria
        """
        return len(self._memory)


    def _chooseAction(self, state):
        """ Obtener la la mejor accion posible dado el estado del ambiente

        Args:
            state (numpy.ndarray): el estado actual del ambiente

        Returns:
            int: la mejor accion posible
        """
        # aleatoriamente regresamos una acción aleatoria para ayudar al
        # agente a que no caiga en un minimo local
        if np.random.rand() <= self._epsilon:
            return random.randrange(self._actionSize)
        # retornamos la mejor accion posible
        QValues = self._QValues.predict(state)
        return np.argmax(QValues[0])


    def _memorize(self, state, action, reward, newState, done):
        """ Alamacenar en la memoria del agente el estado general del ambiente en
            un instante del entrenamiento 

        Args:
            state (numpy.ndarray): estado actual del ambiente
            action (numpy.int64): acción realizada por el agente
            reward (float): recompensa otorgada en base a la acción
            newState (numpy.ndarray): estado nuevo a partir de la acción generada
            done (bool): si el ambiente ha terminado su ejecución o no
        """
        self._memory.append((state, action, reward, newState, done))


    def _load(self, filename):
        """ Cargar una matriz de pesos en el modelo

        Args:
            filename (string): nombre del archivo con la matriz de pesos
        """
        self._QValues.load_weights(filename)


    def _save(self, filename):
        """ Guardar la matriz de pesos actual del modelo

        Args:
            filename (string): nombre del archivo con la matriz de pesos
        """
        self._QValues.save_weights(filename)


    def _learn(self, batchSize):
        """ Entrenar el modelo en base a la memoria acumulada del agente

        Args:
            batchSize (int): porción de la memoria que se empleará para 
                             entrenar el modelo
        """
        # se extraen un batch con los estados más recientes
        batch = random.sample(self._memory, batchSize)

        for state, action, reward, newState, done in batch:
            # empleamos la recompensa del estado general como target para entrenar,
            # esto en caso de que en dicho estado el ambiente siga activo. En caso contrario,
            # se calcula la recompensa correspondiente según la ecuación de Bellman
            target = reward if not done else reward + self._gamma*np.amax(self._QValues.predict(newState)[0])
            # calculamos los Q-values con nuestro modelo actual
            target_f = self._QValues.predict(state)
            # reemplazamos el Q-value correspondiente a la acción generada en el estado general
            # por la recompensa calculada
            target_f[0][action] = target
            # con los Q-values como target, entrenamos nuestro modelos
            self._QValues.fit(state, target_f, epochs=1, verbose=0)


    def learn(self, episodes, timesteps, batchSize=32, save=False):
            """ Someter al agente al proceso de aprendizaje

            Args:
                episodes (int): épocas de entrenamiento
                timesteps (int): instantes de tiempo en los que el ambiente se entrenará/evaluará
                batchSize (int): porción de la memoria que se empleará para entrenar el modelo. Defaults to 32.
                save (bool): flag para indicar al agente si debe o no guardar los pesos. Defaults to False.
            """
            # para cada epoca de entrenamiento
            for episode in range(episodes):
                # obtenemos el estado actual del ambiente y lo redimensionamos
                state = self._env.reset()
                state = self._mapState(state)

                # decaer el exploration rate por episodio
                self._decayEpsilon(episode)

                # durante X pasos generaremos una acción sobre el ambiente
                # y obtendremos una recompensa.
                for t in range(timesteps):
                    # AGENTE EJECUTA ACCIÓN DADO EL ESTADO ACTUAL
                    action = self._chooseAction(state)
                    # obtendremos el estado nuevo del ambiente ante el estimulo dado por el agente
                    # y lo redimencionamos también
                    newState, reward, done, _ = self._env.step(action)
                    newState = self._mapState(newState)
                    # ALMACENAMOS EN MEMORIA DEL AGENTE EL ESTADO GENERAL DEL AMBIENTE 
                    self._memorize(state, action, reward, newState, done)                
                    # actualizamos el estado actual con el nuevo
                    state = newState
                    # si terminó la época, salimos
                    if done:
                        break
                    # SI SE TIENEN LOS DATOS SUFICIENTES, ENTRENAR EL MODELO DEL AGENTE
                    if self._capacity() > batchSize:
                        self._learn(batchSize)
                    # guardar la matriz de pesos cada ciertas épocas
                    if save and episode%10 == 0:
                        self._save(self._path)

                # cerramos el ambiente
                self._env.close()


    def run(self, load=False):
        """ Hacer que el agente interactue con el ambiente

        Args:
            load (bool): flag para indicar al agente si debe o no cargar los pesos. Defaults to False.

        Returns:
            int: intantes de tiempo que duró el agente interactuando con el ambiente
        """
        t = 0
        done = False
        # cargar la matriz de pesos en la NN
        if load:
            self._load(self._path)
        # obtenemos el estado actual del ambiente y lo redimensionamos
        state = self._env.reset()
        state = self._mapState(state)

        while not done:
            self._env.render()
            t = t + 1
            # AGENTE EJECUTA ACCIÓN DADO EL ESTADO ACTUAL
            action = self._chooseAction(state)
            # obtendremos el estado nuevo del ambiente ante el estimulo dado por el agente
            # y lo redimencionamos también
            newState, reward, done, _ = self._env.step(action)
            newState = self._mapState(newState)
            # actualizamos el estado actual con el nuevo
            state = newState

        # cerramos el ambiente
        self._env.close()
        return t

### **Instanciar el agente y entrenarlo**

In [ ]:
# generamos la ruta del folder donde guardaemos los pesos
dirPath = os.path.join(os.path.abspath(''), 'data')
# si no existe el folder, lo creamos
if not os.path.exists(dirPath):
    os.makedirs(dirPath, exist_ok=True)
# observamos la ruta
dirPath

In [ ]:
agent = DQNAgent(environment=gym.make('CartPole-v0'), path=os.path.join(dirPath, 'cartpole-dqn.h5'))

In [ ]:
agent.learn(episodes=10, timesteps=200, save=True)
print('training finished:')

### **Observar al agente en acción**

In [ ]:
t = agent.run(load=True)
t